# 十種猴子影像辨識

## 導入模組

In [59]:
using Flux
using Flux.Data: DataLoader
using Flux: @epochs, Conv, MaxPool, onecold, onehotbatch, throttle, logitcrossentropy
using Augmentor
using Statistics
using ImageView
using Images
using JLD

## 載入資料

In [63]:
pl = Resize(96, 96) |>
     Either(1=>FlipX(), 1=>NoOp()) |>
     Either(1=>Zoom(0.9:0.1:1.2), 1=>NoOp()) |>
     CropSize(96, 96)

4-step Augmentor.ImmutablePipeline:
 1.) Resize to 96×96
 2.) Either: (50%) Flip the X axis. (50%) No operation.
 3.) Either: (50%) Zoom by I ∈ {0.9×0.9, 1.0×1.0, 1.1×1.1, 1.2×1.2}. (50%) No operation.
 4.) Crop a 96×96 window around the center

### 載入訓練資料

In [64]:
monkeys = Array{Float64}(undef, 1097, 96, 96, 3)
mon_key = 1
pbase = "training/training/n"
for i in (0:9)
    stri = string(i)
    p = pbase * stri * "/n" * stri * " ("
    amount = [105, 111, 110, 122, 105, 113, 106, 114, 106, 105]
    for j in (1:amount[i + 1])
        imarray = []
        strj = string(j)
        path = p * strj * ").jpg"
        img = load(path)
        imgg = augment(img, pl)
        mat = channelview(imgg)
        c = copy(mat)
        c2 = zeros(96, 96, 3)
        convert(Array{N0f8, 3}, c2)
        for i in (1:3)
            for j in (1:96)
                for k in (1:96)
                    c2[j, k, i] = copy(convert(N0f8, c[i, j, k]))
                end
            end
        end
        monkeys[mon_key, :, :, :] = c2
        mon_key += 1
    end
end

### 儲存訓練資料

In [65]:
save("monkey_data.jld", "data", monkeys)

### 讀入已儲存的訓練資料

In [66]:
train_x = load("monkey_data.jld")["data"]
train_x = reshape(train_x, 96, 96, 3, :)
train_x = Float32.(train_x)

96×96×3×1097 Array{Float32,4}:
[:, :, 1, 1] =
 0.545098  0.368627   0.0431373   …  0.439216   0.478431   0.32549
 0.160784  0.0        0.623529       0.329412   0.0196078  0.772549
 0.533333  0.6        0.694118       0.388235   0.788235   0.0
 0.470588  0.105882   0.815686       0.541176   0.180392   0.301961
 0.882353  0.137255   0.709804       0.32549    0.317647   0.239216
 0.603922  0.45098    0.137255    …  0.14902    0.913725   0.270588
 0.654902  0.0627451  0.509804       0.678431   0.239216   0.384314
 0.286275  0.462745   0.356863       0.0117647  0.133333   0.435294
 0.572549  0.254902   0.164706       0.921569   0.921569   0.694118
 0.960784  0.0117647  0.0117647      0.419608   0.835294   0.698039
 0.27451   0.321569   0.231373    …  0.337255   0.454902   0.321569
 0.388235  0.517647   0.431373       0.423529   0.419608   0.960784
 0.686275  0.74902    0.352941       0.427451   0.913725   0.419608
 ⋮                                ⋱                        ⋮
 0.6       0.36

### 載入驗證資料

In [67]:
val_monkeys = Array{Float64}(undef, 272, 96, 96, 3)
val_key = 1
pbase = "validation/validation/n"
for i in (0:9)
    stri = string(i)
    p = pbase * stri * "/n" * stri * " ("
    amount = [26, 28, 27, 30, 26, 28, 26, 28, 27, 26]
    for j in (1:amount[i + 1])
        imarray = []
        strj = string(j)
        path = p * strj * ").jpg"
        img = load(path)
        imgg = augment(img, pl)
        mat = channelview(imgg)
        c = copy(mat)
        c2 = zeros(96, 96, 3)
        convert(Array{N0f8, 3}, c2)
        for i in (1:3)
            for j in (1:96)
                for k in (1:96)
                    c2[j, k, i] = copy(convert(N0f8, c[i, j, k]))
                end
            end
        end
        val_monkeys[val_key, :, :, :] = c2
        val_key += 1
    end
end

### 儲存驗證資料

In [68]:
save("monkey_validation.jld", "data", val_monkeys)

### 讀入已儲存的驗證資料

In [69]:
test_x = load("monkey_validation.jld")["data"]
test_x = reshape(test_x, 96, 96, 3, :)
test_x = Float32.(test_x)

96×96×3×272 Array{Float32,4}:
[:, :, 1, 1] =
 0.411765   0.0431373   0.207843   …  0.627451   0.317647   0.117647
 0.411765   0.133333    0.34902       0.0901961  0.501961   0.466667
 0.360784   0.0313726   0.0823529     0.262745   0.654902   0.67451
 0.243137   0.00784314  0.309804      0.223529   0.854902   0.14902
 0.235294   0.580392    0.317647      0.109804   0.231373   0.992157
 0.14902    0.54902     0.74902    …  0.0156863  0.0        0.247059
 0.843137   0.133333    0.596078      0.607843   0.478431   0.54902
 0.792157   0.482353    0.298039      0.392157   0.501961   0.827451
 0.494118   0.223529    0.2           0.321569   0.498039   0.364706
 0.482353   0.270588    0.027451      0.305882   0.537255   0.427451
 0.47451    0.188235    0.352941   …  0.592157   0.572549   0.960784
 0.819608   0.4         0.27451       0.105882   0.0117647  0.227451
 0.764706   0.862745    0.301961      0.282353   0.109804   0.647059
 ⋮                                 ⋱                        ⋮

### 訓練標籤

In [70]:
train_labels = Array{Int32}(undef, 1097)
key = 1
amount = [105, 111, 110, 122, 105, 113, 106, 114, 106, 105]
for i in (0:9)
    for j in (1:amount[i + 1])
        train_labels[key] = i
        key += 1
    end
end
train_y = onehotbatch(train_labels, 0:9)

10×1097 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     1  1  1  1  1  1  1  1  1  1  1  1

### 驗證標籤

In [71]:
val_labels = Array{Int32}(undef, 272)
key = 1
amount = [26, 28, 27, 30, 26, 28, 26, 28, 27, 26]
for i in (0:9)
    for j in (1:amount[i + 1])
        val_labels[key] = i
        key += 1
    end
end
test_y = onehotbatch(val_labels, 0:9)

10×272 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     1  1  1  1  1  1  1  1  1  1  1  1

In [72]:
batchsize = 32
train = DataLoader(train_x, train_y, batchsize = batchsize, shuffle = true)
test = DataLoader(test_x, test_y, batchsize = batchsize)

DataLoader((Float32[0.4117647 0.043137256 … 0.31764707 0.11764706; 0.4117647 0.13333334 … 0.5019608 0.46666667; … ; 0.98039216 0.5529412 … 0.16078432 0.93333334; 0.7137255 0.45490196 … 0.5137255 0.72156864]

Float32[0.5058824 0.73333335 … 0.5254902 0.53333336; 0.43137255 0.02745098 … 0.2901961 0.4; … ; 0.15686275 0.46666667 … 0.7176471 0.1254902; 0.23529412 0.09803922 … 0.34117648 0.43529412]

Float32[0.28235295 0.3254902 … 0.44705883 0.20392157; 0.8980392 0.49803922 … 0.45490196 0.3019608; … ; 0.3019608 0.2784314 … 0.8627451 0.69803923; 0.54901963 0.4745098 … 0.0 0.5294118]

Float32[0.11764706 0.3019608 … 0.6627451 0.37254903; 0.36078432 0.8156863 … 0.05490196 0.38039216; … ; 0.5568628 0.9843137 … 0.25490198 0.16470589; 0.7490196 0.77254903 … 0.4 0.81960785]

Float32[0.13333334 0.39607844 … 0.5176471 0.94509804; 0.38039216 0.07450981 … 0.4862745 0.4; … ; 0.9490196 0.17254902 … 0.75686276 0.49803922; 0.5882353 0.2509804 … 0.73333335 0.34509805]

Float32[0.60784316 0.6039216 … 0.2392156

## CNN 模型

In [82]:
model_easy = Chain(
    
    BatchNorm(3),
    Conv((3, 3), 3 => 64, pad = (1, 1), relu), # 96x96
    MaxPool((2, 2), stride = (2, 2)), # 48x48

    BatchNorm(64),
    Conv((3, 3), 64 => 128, pad = (1, 1), relu), # 48x48
    MaxPool((2, 2), stride = (2, 2)), # 24x24

    BatchNorm(128),
    Conv((3, 3), 128 => 64, pad = (1, 1), relu), # 24x24
    MaxPool((2, 2), stride = (2, 2)), # 12x12

    BatchNorm(64),
    Conv((3, 3), 64 => 64, pad = (1, 1), relu), # 12x12
    MaxPool((2, 2), stride = (2, 2)), # 6x6
    
    Flux.flatten,
    Dense(2304, 256, relu),
    BatchNorm(256),
    Dropout(0.4f0),
    Dense(256, 10),
    softmax
)

Chain(BatchNorm(3), Conv((3, 3), 3=>64, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), BatchNorm(64), Conv((3, 3), 64=>128, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), BatchNorm(128), Conv((3, 3), 128=>64, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), BatchNorm(64), Conv((3, 3), 64=>64, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), flatten, Dense(2304, 256, relu), BatchNorm(256), Dropout(0.4), Dense(256, 10), softmax)

In [83]:
size(train_x)

(96, 96, 3, 1097)

In [84]:
model_easy(train_x[:, :, :, 1:32])

10×32 Array{Float32,2}:
 0.0847707  0.0831225  0.0827994  …  0.0833515  0.083511   0.083957
 0.106855   0.105585   0.104979      0.105969   0.105734   0.106122
 0.10296    0.101774   0.101809      0.100506   0.101917   0.101799
 0.0972499  0.0975377  0.0980651     0.0977677  0.0970493  0.097539
 0.100283   0.10408    0.102734      0.101816   0.103754   0.102293
 0.107765   0.106647   0.107402   …  0.106329   0.105486   0.108096
 0.0881328  0.0877692  0.0882001     0.0905776  0.0880316  0.0869207
 0.100947   0.102261   0.100636      0.101337   0.100061   0.100426
 0.109329   0.106682   0.107896      0.108455   0.108833   0.108435
 0.101707   0.104542   0.105479      0.10389    0.105623   0.104412

## 損失函數

In [85]:
loss(x, y) = logitcrossentropy(model_easy(x), y)

loss (generic function with 1 method)

In [86]:
accuracy(x, y, model) = mean(onecold(model_easy(x)) .== onecold(y))

accuracy (generic function with 1 method)

## Callback 函式

In [87]:
function callback()
    print("Testing ")
    @show loss(test_x, test_y)
    print("Testing ")
    @show accuracy(test_x, test_y, model_easy)
    println()
    accuracy(test_x, test_y, model_easy) > 0.4 && Flux.stop()
end

callback (generic function with 1 method)

## 模型訓練

In [88]:
num_epochs = 10
lr_decay = 0.01
lr = 0.002
for i in (1:num_epochs)
    @epochs 1 Flux.train!(loss, params(model_easy), train, 
                          ADAM(lr / (1 + lr_decay ^ (i - 1)), (0.9, 0.8)), 
                          cb = throttle(callback, 500))
end

┌ Info: Epoch 1
└ @ Main C:\Users\user\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


Testing loss(test_x, test_y) = 2.3028233f0
Testing accuracy(test_x, test_y, model_easy) = 0.08823529411764706



┌ Info: Epoch 1
└ @ Main C:\Users\user\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


Testing loss(test_x, test_y) = 2.275121f0
Testing accuracy(test_x, test_y, model_easy) = 0.17647058823529413



┌ Info: Epoch 1
└ @ Main C:\Users\user\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


...

Testing loss(test_x, test_y) = 2.2582352f0
Testing accuracy(test_x, test_y, model_easy) = 0.18382352941176472



┌ Info: Epoch 1
└ @ Main C:\Users\user\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


Testing loss(test_x, test_y) = 2.1542342f0
Testing accuracy(test_x, test_y, model_easy) = 0.3088235294117647



┌ Info: Epoch 1
└ @ Main C:\Users\user\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


Testing loss(test_x, test_y) = 2.24857f0
Testing accuracy(test_x, test_y, model_easy) = 0.20220588235294118



## 模型評估

In [89]:
print("Validation Accuracy: $(accuracy(test_x, test_y, model_easy) * 100)%")

Validation Accuracy: 31.25%